# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Описание переменных

Описание переменных:
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита.

In [88]:
!pip install pymystem3

In [89]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
m = Mystem()

In [90]:
df = pd.read_csv('project6.csv')
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [91]:
print('Количество строк -- {}, количество колонок -- {}'.
     format(df.shape[0], df.shape[1]))

Количество строк -- 21525, количество колонок -- 12


In [92]:
print('Количество дублирующихся записей -- {}. Это составляет -- {:.2%} от общего числа записей.'.
      format(df.duplicated().sum(), df.duplicated().sum()/df.shape[0]))

Количество дублирующихся записей -- 54. Это составляет -- 0.25% от общего числа записей.


In [93]:
df.drop_duplicates(inplace=True)

In [94]:
print('Количество дублирующихся записей -- {}. Это составляет -- {:.2%} от общего числа записей.'.
      format(df.duplicated().sum(), df.duplicated().sum()/df.shape[0]))

Количество дублирующихся записей -- 0. Это составляет -- 0.00% от общего числа записей.


In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21471 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21471 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21471 non-null  int64  
 3   education         21471 non-null  object 
 4   education_id      21471 non-null  int64  
 5   family_status     21471 non-null  object 
 6   family_status_id  21471 non-null  int64  
 7   gender            21471 non-null  object 
 8   income_type       21471 non-null  object 
 9   debt              21471 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21471 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


In [96]:
df.isna().sum()

children               0
days_employed       2120
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2120
purpose                0
dtype: int64

In [97]:
df[df.days_employed.isna()].index

Int64Index([   12,    26,    29,    41,    55,    65,    67,    72,    82,
               83,
            ...
            21414, 21423, 21426, 21432, 21463, 21489, 21495, 21497, 21502,
            21510],
           dtype='int64', length=2120)

In [98]:
df[df.total_income.isna()].index

Int64Index([   12,    26,    29,    41,    55,    65,    67,    72,    82,
               83,
            ...
            21414, 21423, 21426, 21432, 21463, 21489, 21495, 21497, 21502,
            21510],
           dtype='int64', length=2120)

Так как пропусков в признаках `days_employed` и `total_income` не много (примерно 10% от всей выборки) и будет не целеобразно их как-то заменять (нам нужна более точная статистика), то считаю, что данные с пропусками надо удалить. К тому же, судя по индексам, пропуски присутствуют в одинаковых индексах

In [99]:
df.drop(df[df.days_employed.isna()].index, inplace=True)

In [100]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [101]:
dict_to_rus = {
'children' : 'количество детей в семье',
'days_employed' : 'общий трудовой стаж в днях',
'dob_years' : 'возраст клиента в годах',
'education_id' : 'идентификатор уровня образования',
'family_status_id' : 'идентификатор семейного положения',
'debt' : 'имел ли задолженность по возврату кредитов',
'total_income' : 'ежемесячный доход'
}

In [102]:
pd.concat([pd.DataFrame(list(dict_to_rus.values()), index = list(dict_to_rus.keys()), columns = ['расшифровка']), 
           df.describe().T], axis = 1, sort = False)

,расшифровка,count,mean,std,min,25%,50%,75%,max
children,количество детей в семье,19351.0,0.537388,1.371408,-1.000000,0.000000,0.000000,1.000000,2.000000e+01
days_employed,общий трудовой стаж в днях,19351.0,63046.497661,140827.311974,-18388.949901,-2747.423625,-1203.369529,-291.095954,4.017554e+05
dob_years,возраст клиента в годах,19351.0,43.255336,12.579170,0.000000,33.000000,42.000000,53.000000,7.500000e+01
education_id,идентификатор уровня образования,19351.0,0.819079,0.550104,0.000000,1.000000,1.000000,1.000000,4.000000e+00
family_status_id,идентификатор семейного положения,19351.0,0.972249,1.420596,0.000000,0.000000,0.000000,1.000000,4.000000e+00
debt,имел ли задолженность по возврату кредитов,19351.0,0.081184,0.273125,0.000000,0.000000,0.000000,0.000000,1.000000e+00
total_income,ежемесячный доход,19351.0,167422.302208,102971.566448,20667.263793,103053.152913,145017.937533,203435.067663,2.265604e+06


In [103]:
df.describe(include=[object])

,education,family_status,gender,income_type,purpose
count,19351,19351,19351,19351,19351
unique,15,5,3,8,38
top,среднее,женат / замужем,F,сотрудник,свадьба
freq,12342,11143,12752,10014,721


**Выводы по каждой из имеющихся переменных:**
1. Переменная `children` --- <<Количество детей в семье>>. Имеет верный целочисленный тип. Пропусков данная переменная не содержит. Однако, минимальное количество детей равно `-1`(!), а максиммальное `20`(!), при этом другие статистики показывают, что у большинства клиентов только 1 ребенок. В связи с этим далее посмотреть на количество записей с отрицательным и большим числом детей. Следуя совету одного из уроков, в реальной ситуации подобные записи, например, с отрицательным числом детей, следовало бы уточнить у заказчика. Возможно значение `-1` это `дефис 1`, т.е. ошибка ввода.
2. Переменная `days_employed` --- <<Общий трудовой стаж в днях>>. Переменная вещественного типа, имеются отрицательные и дробные значения --- явные ошибки. Имеются пропуски. Если перевести максимальное значение переменной в годы получим `401755.400475/365 =1101`, т.е. человек работал более 1000!!! лет. Представляется, что дальнейшая обработка данной переменной потребует значительных усилий, о чем также свидетельствуют и значения статистик.
3. Переменная `dob_years` --- <<Возраст клиента в годах>>. Имеет верный целочисленный тип, пропусков нет. Минимальное значение переменной `0` --- что подозрительно. Однако в целом замечаний к переменной нет.
4. Переменные `education` --- <<Уровень образования клиента>> и `education_id` --- <<Идентификатор уровня образования>>. Пропусков нет. В столбце `education` одни и те же значения написаны прописными и строчными буквами --- требует преобразования. Переменные являются категориальными (представлены строкой и числом, соответственно), которые, по сути, несут одну и ту же информацию о клиенте --- наличие определенного уровня образования. Данный факт говорит в пользу исключения одного из столбцов при дальнейшем рассмотрении.
5. Переменные `family_status` --- <<Семейное положение>> и  `family_status_id` --- <<Идентификатор семейного положения>>. Ситуация аналочиная переменным с образованием. Статистики говорят, что в выборке больше семейных.
6. Переменные `gender` --- <<Пол клиента>> и `income_type` --- <<Тип занятости>>. Категориальные переменные, записанные через стровый тип, пропусков нет. Замечаний нет, хотя возможны строчные и пропускные буквы, а также иные записи, быстро проверить методом `.value_counts()`.   
7. Переменная `debt` --- <<Имел ли задолженность по возврату кредитов>>. Целочисленный тип. Переменная категориальная, принимающая значения из множества $\{0,1\}$. Представляется целесообразным изменить ее тип на логический. Пропусков нет. Судя по статистике, большинство клиентов не имели задолженностей. 
8. Переменная `total_income` --- <<Ежемесячный доход>>. Вещественная переменная -- нормально. Имеются пропуски. Вычисленные статистики не позволяют сделать вывод о наличии в данных аномалий. Предполагается, что должна быть связь между `total_income` и `days_employed`. Возможно наличие очень больших доходов. 
9. Переменная `purpose` --- <<Цель получения кредита>>. Строковый тип, пропусков нет. Значения переменной содержат близкие по смыслу формулировки, например, `покупка жилья` и `операции с жильем`; `на покупку своего автомобиля` и `на покупку автомобиля`. Видимо именно эта переменная нуждается в лемматизации. После этого возможно удастся выделить категории этой переменной. 

**На этом исчерпывается информация, которую удалось извлечь из просмотра набора данных и анализа описательных статистик.** 

Посмотрим на аномалии с количеством детей

In [104]:
df.children.value_counts()

 0     12710
 1      4343
 2      1851
 3       294
 20       67
-1        44
 4        34
 5         8
Name: children, dtype: int64

Как мы видим, что с 20 детьми аж целых 67 человек, что говорит о том, что это явно не уникальный случай, а какая-то ошибка, либо опечатка. Поэтому вглянем на этих людей и решим на какую цифру была опечатка. на 2 или на 0. Так же видим, что у 44 людей -1 ребёнок, что в реальности невозможно. Тут возникает опять же вопрос. Опечатка или же какая-то ошибка? Поэтому так же взглянем на этих людей и возьмём за основу их семейное положение.

In [105]:
df.loc[df['children'] == 20].family_status.value_counts()

женат / замужем          46
гражданский брак         11
Не женат / не замужем     6
вдовец / вдова            3
в разводе                 1
Name: family_status, dtype: int64

Как мы можем видеть, что больше всех у женатых, либо же у тех кто в гражданском браке, что говорит о том, что скорее всего была опечатка на 0, а не на 2. Поэтому для этой группы приравняем колличество детей к 2, а не к 0. Теперь перейдём к следующей группе и посмотрим на неё. Так же за наш показатель возьмём их семейное положение.

In [106]:
df.loc[df['children'] == -1].family_status.value_counts()

женат / замужем          27
гражданский брак          5
Не женат / не замужем     4
в разводе                 4
вдовец / вдова            4
Name: family_status, dtype: int64

Тут, практически, такая же картина. Так что приравняем наш показатель к положительному числу.

In [107]:
df.children = df.children.abs()
df.loc[df['children'] == 20, 'children'] = 2
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,19351.000000,19351.000000,19351.000000,19351.000000,19351.000000,19351.000000,1.935100e+04
mean,0.479613,63046.497661,43.255336,0.819079,0.972249,0.081184,1.674223e+05
std,0.753895,140827.311974,12.579170,0.550104,1.420596,0.273125,1.029716e+05
min,0.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,5.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Теперь всё достаточно понятно с детьми и показатели лежат в пределах реального.

Далее перейдём к следующему вопоросу и посмотрим, что не так у нас возрастом. Откуда взялся минимальный показатель 0 и как часто он всречается в нашей выборке. Но раз у нас есть нулевые значения, то возможно, есть и другие неверные показатели. Давайте вглянем на них.

In [108]:
df.query('dob_years < 17').dob_years.value_counts()

0    91
Name: dob_years, dtype: int64

Как видим, что у нас всего 1 единственный неверный показатель, что говорит о том, что человек скорее всего не указал свой возраст, либо случайно пропустил при заполнении формы. Поэтому заменим нулевые значения на медианные значения по каждому виду занятости.

In [109]:
df.loc[df['dob_years'] == 0, 'dob_years'] = df.groupby('income_type')['dob_years'].transform('median')
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,19351.000000,19351.000000,19351.000000,19351.000000,19351.000000,19351.000000,1.935100e+04
mean,0.479613,63046.497661,43.457496,0.819079,0.972249,0.081184,1.674223e+05
std,0.753895,140827.311974,12.235554,0.550104,1.420596,0.273125,1.029716e+05
min,0.000000,-18388.949901,19.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,5.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Выбросы почистили. Все в пределах нормы. Осталось обработать количество дней стажа.

In [110]:
df.days_employed.loc[df['days_employed'] < 0].min() #т.к. имеем дело с отрицательными числами,
                                                    #то данный показатель будет показывать максимальное значение

-18388.949900568383

In [111]:
df.days_employed.loc[df['days_employed'] < 0].max() #т.к. имеем дело с отрицательными числами, 
                                                    #то данный показатель будет показывать минимальное значение

-24.14163324048118

In [112]:
df.days_employed.loc[df['days_employed'] < 0].median()

-1630.0193809778218

Как видим, что практически 2/3 стоблца имеют отрицательные показатели, а так же минимальный опыт работы похож на реальный показатель, так же как и максмальное значение не уходит в запредельные числа, так что это говорит о том что вряд ли это ошибка и скорее всего обратный отсчёт от даты выгрузки. Так же можем заметить, что медиана равна ~ 4,5 года, что говорит о том, что стаж работы указан на одном(действительном рабочем месте) отсюда можно и связать, что отрицательные значения возникают от даты выгрузки датасета. Т.е. при заполнения формы человек указал, что работает по настоящее время. Для того, чтобы подвердить нашу теорию обратимся к человеку с минимальным опытом работы

In [113]:
empl = df.query('days_employed == -24.14163324048118')
empl

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
17437,1,-24.141633,31,среднее,1,женат / замужем,0,F,сотрудник,1,166952.415427,высшее образование


Обратим внимание, что данному человеку на данный момент 31 год и имеет среднее образование, так же что есть ребёнок и это женский пол. Возможно, человек не работал всю свою жизнь и была в декрете/домохозяйкой, но вспомним значение медианы и оставим нашу мысль на потом для других исследований.

Теперь посмотрим положительные показатели по отработанным дням и так же минимальное и максимальное значение

In [114]:
df.days_employed.loc[df['days_employed'] > 0].count()

3445

In [115]:
df.days_employed.loc[df['days_employed'] > 0].min()

328728.72060451825

In [116]:
df.days_employed.loc[df['days_employed'] > 0].max()

401755.40047533

In [117]:
df.days_employed.loc[df['days_employed'] > 0].median()

365213.3062657312

не так уж и много. По сравнению с отрицательными значениями. Но минимальные и максимальные значения уходят в заоблачные показатели, так же как и медиана. Попробуем найти взаимосвязь среди вида занятости.

In [118]:
positive_days_employed = df.query('days_employed > 0').income_type.value_counts()
positive_days_employed

пенсионер      3443
безработный       2
Name: income_type, dtype: int64

In [119]:
negative_days_employed = df.query('days_employed < 0').income_type.value_counts()
negative_days_employed

сотрудник          10014
компаньон           4577
госслужащий         1312
студент                1
предприниматель        1
в декрете              1
Name: income_type, dtype: int64

Отрицательные показатели отображаются у тех, кто на данный момент имеет трудовую занятость, либо так же имеет, но находится в отпуске(декрет), так же как и студент может иметь какую-либо трудовую деятельность(подработку). т.е. простыми словами, days_employed отображает сколько человек работает на данный момент на данной должности, т.е. это показатель не общего стажа работы, а только стажа его нынешней работы, отсюда же и следует вывод, что положительные значения ошибочны и находятся в значениях у тех, кто сейчас не работает на данный момент, следовательно, все данные можно приравнять к нулю для корректных отображений наших данных.

Для этого возьмём пенсионеров и безработных и укажем, что их стаж равен 0.

In [120]:
df.loc[df['days_employed'] > 0, 'days_employed'] = 0

Теперь после того как мы привели наши данные к более реалистичным показателям, приведём наши отрицательные числа к положительным.

In [121]:
df.days_employed = df.days_employed.abs()

In [122]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,19351.000000,19351.000000,19351.000000,19351.000000,19351.000000,19351.000000,1.935100e+04
mean,0.479613,1934.115623,43.457496,0.819079,0.972249,0.081184,1.674223e+05
std,0.753895,2274.751213,12.235554,0.550104,1.420596,0.273125,1.029716e+05
min,0.000000,0.000000,19.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,291.095954,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,2747.423625,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,5.000000,18388.949901,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Замена типа данных

- переменная `days_employed` имеет вещественный тип, а **надо** сделать целочисленный;
- переменная `debt` имеет целочисленный тип, а **желательно** сделать логический.

Причина изменения типа для переменной `days_employed` очевидна --- общий трудовой стаж в днях должно быть целым числом. 
Изменить тип переменной `debt` на логический желательно. Переменная принимает всего два значения 0 или 1.

In [123]:
df.astype({'days_employed': 'int64', 'debt': 'bool'}).dtypes

children              int64
days_employed         int64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                   bool
total_income        float64
purpose              object
dtype: object

Посмотрим уникальные значения признаков...

In [124]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [125]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [126]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'получение образования',
       'сделка с подержанным автомобилем', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее образование',
       'покупка жилья для сдачи

$\bullet$ Приведение значений исследуемых переменных к нижнему регистру

In [127]:
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()

In [128]:
df.education.value_counts()

среднее                13693
высшее                  4716
неоконченное высшее      675
начальное                261
ученая степень             6
Name: education, dtype: int64

In [129]:
df.family_status.value_counts()

женат / замужем          11143
гражданский брак          3735
не женат / не замужем     2525
в разводе                 1083
вдовец / вдова             865
Name: family_status, dtype: int64

In [130]:
df['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [131]:
df['gender'].value_counts()

F      12752
M       6598
XNA        1
Name: gender, dtype: int64

In [132]:
df.drop(df[df.gender == 'XNA'].index, inplace=True)
df['gender'].unique()

array(['F', 'M'], dtype=object)

### Лемматизация

Для дальнейшей категоризации данных воспользуемся лемматизацией слов в столбце "purpose". Для этого импортируем нужную нам библиотеку, но сначала взглянем на этот столбец. Иморт библиотеки запишем в начало, там где хранятся остальные библиотеки.

In [133]:
df.purpose.value_counts()

свадьба                                   721
сыграть свадьбу                           693
на проведение свадьбы                     685
операции с недвижимостью                  615
покупка коммерческой недвижимости         597
покупка жилья                             595
покупка жилья для сдачи                   588
жилье                                     587
операции с коммерческой недвижимостью     581
строительство жилой недвижимости          580
операции с жильем                         579
покупка своего жилья                      574
недвижимость                              572
покупка жилья для семьи                   570
строительство недвижимости                561
строительство собственной недвижимости    560
операции со своей недвижимостью           559
покупка недвижимости                      551
покупка жилой недвижимости                546
ремонт жилью                              542
автомобиль                                454
на покупку своего автомобиля      

Как видим, что очень много одинаковых по смыслу слов, но написаны они по-разному. Для того, чтобы в дальнейшем категоризировать их - лемматизируем их и взглянем ещё раз.

In [134]:
df['category_purpose'] = df.purpose.apply(m.lemmatize)

In [135]:
df.category_purpose.value_counts()

[автомобиль, \n]                                          875
[свадьба, \n]                                             721
[сыграть,  , свадьба, \n]                                 693
[на,  , проведение,  , свадьба, \n]                       685
[операция,  , с,  , недвижимость, \n]                     615
[покупка,  , коммерческий,  , недвижимость, \n]           597
[покупка,  , жилье, \n]                                   595
[покупка,  , жилье,  , для,  , сдача, \n]                 588
[жилье, \n]                                               587
[операция,  , с,  , коммерческий,  , недвижимость, \n]    581
[строительство,  , жилой,  , недвижимость, \n]            580
[операция,  , с,  , жилье, \n]                            579
[покупка,  , свой,  , жилье, \n]                          574
[недвижимость, \n]                                        572
[покупка,  , жилье,  , для,  , семья, \n]                 570
[строительство,  , недвижимость, \n]                      561
[строите

In [136]:
def change(column):
    for row in column:
        if 'автомобиль' in row:
            return 'автомобиль'
        if 'свадьба' in row:
            return 'свадьба'
        if 'образование' in row:
            return 'образование'
        if 'жилье' in row:
            return 'недвижимость'
        if 'недвижимость' in row:
            return 'недвижимость'

In [137]:
df.category_purpose = df.category_purpose.apply(change)

In [138]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,category_purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,недвижимость
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,недвижимость
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,образование
4,0,0.000000,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,свадьба
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,недвижимость
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,недвижимость
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,образование
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,свадьба
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,недвижимость


In [139]:
df.category_purpose.value_counts()

недвижимость    9757
автомобиль      3897
образование     3597
свадьба         2099
Name: category_purpose, dtype: int64

In [140]:
df.category_purpose.isnull().sum()

0

Отлично! Данные почищены и пригодны для анализа.

### Категоризация данных

Теперь для того, чтобы ответить на поставленные вопросы, мы можем приступить к категоризации данных. Исходя из нашей таблицы мы можем разбить людей по:

1. доходам
2. наличию детей
3. задолженности

Разобьём сначала людей по доходам. За основу возьмём данные Росстата, где сказано, что:

1. бедные - < 17 000
2. среднедостаточные - от 17 000 до 50 000
3. состоятельные - от 50000 до 100 000
4. богатые - от 100 000 до 500 000
5. сверхбогатые - свыше 500 000

In [141]:
def name_social_class(row):
    income = row['total_income']
    if income < 17000:
        return 'бедные'
    if 17000 <= income <= 50000:
        return 'среднедостаточные'
    if 50001 <= income <= 100000:
        return 'состоятельные'
    if 100001 <= income <= 500000:
        return 'богатые'
    if income > 500000:
        return 'бедные'
    
df['social_status'] = df.apply(name_social_class, axis = 1)
df['social_status'].value_counts()

богатые              14665
состоятельные         4091
среднедостаточные      372
бедные                 222
Name: social_status, dtype: int64

Теперь разобьём людей на группу по колличеству детей. В России считается что:

1. 1-2 ребёнка - традиционная семья
2. больше 2 детей - многодетная семья
3. нет детей - бездетная семья

Напишем функцию, которая определит какой гражданин относится к какой категории.

In [143]:
def count_kids(row):
    kid = row['children']
    if kid < 1:
        return 'нет детей'
    if kid >= 1:
        return 'есть дети'


df['type_of_family'] = df.apply(count_kids, axis = 1)
df['type_of_family'].value_counts()

нет детей    12709
есть дети     6641
Name: type_of_family, dtype: int64

**Вывод**

В данном блоке мы завершили финальную подготовку данных, а именно категоризацию данных, и готовы перейти к ответам на вопрос банка.

Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [144]:
def debts(row):
    deb = row['debt']
    if deb == 1:
        return 'должен'
    if deb == 0:
        return 'не должен'

df['actual_debt'] = df.apply(debts, axis = 1)
df['actual_debt'].value_counts()

не должен    17779
должен        1571
Name: actual_debt, dtype: int64

In [145]:
df.pivot_table('debt', ['type_of_family', 'actual_debt'], aggfunc = 'count')

debt
type_of_family actual_debt       
есть дети      должен         619
               не должен     6022
нет детей      должен         952
               не должен    11757

In [146]:
df.pivot_table(index='type_of_family', values='debt', aggfunc=['count','sum','mean'])

,count,sum,mean
,debt,debt,debt
type_of_family,,,
есть дети,6641,619,0.093209
нет детей,12709,952,0.074908


In [149]:
df.groupby('type_of_family', as_index=False).agg({'debt':'mean'}).sort_values(by='debt', ascending=False)

,type_of_family,debt
0,есть дети,0.093209
1,нет детей,0.074908


Как видим, что те люди у которых нет детей более ответственны к своим долговым обязательствам, чем те, у которых они есть.

И следующий вопрос...

Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [150]:
df.pivot_table('debt', ['family_status', 'actual_debt'], aggfunc = 'count')

debt
family_status         actual_debt       
в разводе             должен          76
                      не должен     1007
вдовец / вдова        должен          56
                      не должен      809
гражданский брак      должен         339
                      не должен     3395
женат / замужем       должен         846
                      не должен    10297
не женат / не замужем должен         254
                      не должен     2271

In [151]:
df.groupby('family_status', as_index=False).agg({'debt':'mean'}).sort_values(by='debt', ascending=False)

,family_status,debt
4,не женат / не замужем,0.100594
2,гражданский брак,0.090787
3,женат / замужем,0.075922
0,в разводе,0.070175
1,вдовец / вдова,0.064740


Люди, кто не имеет законных отношений более склонны к задолжности.

Ответим еще на один вопрос.

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [152]:
df.pivot_table('debt', ['social_status', 'actual_debt'], aggfunc = 'count')

debt
social_status     actual_debt       
бедные            должен          14
                  не должен      208
богатые           должен        1203
                  не должен    13462
состоятельные     должен         331
                  не должен     3760
среднедостаточные должен          23
                  не должен      349

In [153]:
df.groupby('social_status', as_index=False).agg({'debt':'mean'}).sort_values(by='debt', ascending=False)

,social_status,debt
1,богатые,0.082032
2,состоятельные,0.080909
0,бедные,0.063063
3,среднедостаточные,0.061828


Как видим, что больше всего в нашей выборке богатых людей, но так же, что у них самый высокий показатель задолженности.

И последний интересующий вопрос...

Как разные цели кредита влияют на его возврат в срок?

In [154]:
df.pivot_table('debt', ['category_purpose', 'actual_debt'], aggfunc = 'count')

debt
category_purpose actual_debt      
автомобиль       должен        367
                 не должен    3530
недвижимость     должен        715
                 не должен    9042
образование      должен        331
                 не должен    3266
свадьба          должен        158
                 не должен    1941

In [155]:
df.groupby('category_purpose', as_index=False).agg({'debt':'mean'}).sort_values(by='debt', ascending=False)

,category_purpose,debt
0,автомобиль,0.094175
2,образование,0.092021
3,свадьба,0.075274
1,недвижимость,0.073281


Исходя из полученных данных можно увидеть, что самыми надежными заёмщиками являются те люди, чья цель кредита является в приобретении недвижимости и самые ненадёжные те, кто берёт кредит с целью свадьбы и образования.

## Общий вывод

1. Те люди у которых нет детей более ответственны к своим долговым обязательствам, чем те, у которых они есть.
2. Те люди, кто не имеет законных отношений более склонны к задолжности.
3. Чем ниже достаток, тем более ответственен заёмщик
4. Те люди, чья цель кредита является в приобретении недвижимости являются самыми надёжными, чем те чья цель является свадьба или же образование

## Заключение

Получив ответы на вопрос, мы можем сделать вывод, что самые надежные заёмщики те, кто:

1. не имеет детей, его достаток составляет не выше среднего, в законных отношениях, либо имел их и цель их покупки - недвижимость

Соответственно, можно сделать вывод, что самые ненадежные заёмщики те, кто:

1. имеет детей, имеет хороший достаток, не имеет законных отношений или не был в них, цель покупки - образование или автомобиль.